# Queries & Filters

**Use case:** Building an admin dashboard with complex user search.

This notebook covers the QuerySet API -- filtering, ordering, pagination,
field selection, and Q objects for complex conditions. If you know Django's
QuerySet, you will feel right at home.

## Prerequisites

A SurrealDB Docker container must be running. See `00_setup.ipynb` for details.

In [ ]:
# -- Setup -----------------------------------------------------------------
import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)
print("Connection configured.")

## Define a User Model and Seed Sample Data

We'll create a handful of users with different roles, ages, and metadata
so we have something interesting to query.

In [ ]:
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict


class User(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="users")

    id: str | None = None
    name: str
    email: str
    role: str = "member"
    age: int = 0
    is_verified: bool = False
    tags: list[str] = []


# Seed data ----------------------------------------------------------------
users_data = [
    User(name="Alice",   email="alice@example.com",   role="admin",     age=32, is_verified=True,  tags=["python", "devops"]),
    User(name="Bob",     email="bob@example.com",     role="member",    age=25, is_verified=True,  tags=["javascript"]),
    User(name="Charlie", email="charlie@example.com", role="member",    age=17, is_verified=False, tags=["python", "rust"]),
    User(name="Diana",   email="diana@example.com",   role="admin",     age=45, is_verified=True,  tags=["devops", "go"]),
    User(name="Eve",     email="eve@example.com",     role="moderator", age=28, is_verified=False, tags=["python"]),
    User(name="Frank",   email="frank@example.com",   role="member",    age=19, is_verified=True,  tags=[]),
]

for u in users_data:
    await u.save()

print(f"Seeded {len(users_data)} users.")

## Basic Filters -- `filter()`

The `filter()` method takes keyword arguments in the form `field__lookup=value`.
The default lookup is `exact` (equality). Filters are chainable -- each call
adds an AND condition.

In [ ]:
# Exact match -- find all admins
admins = await User.objects().filter(role="admin").exec()
print(f"Admins ({len(admins)}):")
for u in admins:
    print(f"  - {u.name} (age {u.age})")

# Greater-than-or-equal -- find users aged 18+
adults = await User.objects().filter(age__gte=18).exec()
print(f"\nAdults ({len(adults)}):")
for u in adults:
    print(f"  - {u.name} (age {u.age})")

## All Filter Lookups

The ORM supports a wide range of lookups, all translated to valid SurrealQL
functions or operators. Filter values are automatically parameterized
(`$_f0`, `$_f1`, ...) to prevent injection.

| Lookup          | SurrealQL Generated                      | Example                                |
|-----------------|------------------------------------------|----------------------------------------|
| `exact` (default) | `=`                                   | `filter(role="admin")`                 |
| `gt`            | `>`                                      | `filter(age__gt=30)`                   |
| `gte`           | `>=`                                     | `filter(age__gte=18)`                  |
| `lt`            | `<`                                      | `filter(age__lt=30)`                   |
| `lte`           | `<=`                                     | `filter(age__lte=25)`                  |
| `in`            | `IN`                                     | `filter(role__in=["admin","mod"])`    |
| `not_in`        | `NOT IN`                                 | `filter(role__not_in=["banned"])`     |
| `contains`      | `CONTAINS`                               | `filter(tags__contains="python")`      |
| `icontains`     | `string::contains(string::lowercase())` | `filter(name__icontains="alice")`     |
| `not_contains`  | `CONTAINSNOT`                            | `filter(tags__not_contains="go")`      |
| `containsall`   | `CONTAINSALL`                            | `filter(tags__containsall=[...])`      |
| `containsany`   | `CONTAINSANY`                            | `filter(tags__containsany=[...])`      |
| `startswith`    | `string::starts_with()`                  | `filter(name__startswith="A")`         |
| `endswith`      | `string::ends_with()`                    | `filter(email__endswith=".com")`       |
| `like`          | `string::matches()` (LIKE→regex)         | `filter(name__like="%ali%")`           |
| `ilike`         | `string::matches()` with `(?i)`          | `filter(name__ilike="%ali%")`          |
| `regex`         | `string::matches()`                      | `filter(name__regex="gr(a\|e)y")`      |
| `iregex`        | `string::matches()` with `(?i)`          | `filter(name__iregex="hello")`         |
| `match`         | `@@` (full-text search)                  | `filter(title__match="quantum")`       |
| `isnull`        | `IS NULL` / `IS NOT NULL`                | `filter(email__isnull=False)`          |

In [ ]:
# IN lookup -- users who are admin or moderator
privileged = await User.objects().filter(role__in=["admin", "moderator"]).exec()
print(f"Privileged roles: {[u.name for u in privileged]}")

# CONTAINS -- users whose tags include "python"
pythonistas = await User.objects().filter(tags__contains="python").exec()
print(f"Python users:     {[u.name for u in pythonistas]}")

# CONTAINSANY -- users with at least one of these tags
devops_or_go = await User.objects().filter(tags__containsany=["devops", "go"]).exec()
print(f"DevOps or Go:     {[u.name for u in devops_or_go]}")

# STARTSWITH -- names starting with a letter
d_names = await User.objects().filter(name__startswith="D").exec()
print(f"Names starting D: {[u.name for u in d_names]}")

# NOT_IN -- exclude members
not_members = await User.objects().filter(role__not_in=["member"]).exec()
print(f"Not members:      {[u.name for u in not_members]}")

# Chained filters (AND) -- verified adults
verified_adults = await User.objects().filter(is_verified=True).filter(age__gte=18).exec()
print(f"Verified adults:  {[u.name for u in verified_adults]}")

## Q Objects -- Complex Conditions

When you need OR logic, negation, or deeply nested conditions, use `Q` objects.
They support three operators:

- `|` (OR)  -- `Q(a=1) | Q(b=2)`
- `&` (AND) -- `Q(a=1) & Q(b=2)`
- `~` (NOT) -- `~Q(a=1)`

You can mix Q objects with regular keyword arguments in a single `filter()` call.

In [ ]:
from src.surreal_orm import Q

# OR -- users named Alice OR whose email contains "bob"
result = await User.objects().filter(
    Q(name="Alice") | Q(email__contains="bob"),
).exec()
print(f"Alice or Bob email: {[u.name for u in result]}")

# AND + OR -- admins who are either 30+ or verified
result = await User.objects().filter(
    Q(role="admin") & (Q(age__gte=30) | Q(is_verified=True)),
).exec()
print(f"Admin & (30+ or verified): {[u.name for u in result]}")

# NOT -- everyone except members
result = await User.objects().filter(~Q(role="member")).exec()
print(f"Not members: {[u.name for u in result]}")

# Mixed: Q objects + regular kwargs
# Search for "ali" in name or email, limited to verified users
result = await User.objects().filter(
    Q(name__ilike="%ali%") | Q(email__ilike="%ali%"),
    is_verified=True,
).exec()
print(f"Search 'ali' (verified only): {[u.name for u in result]}")

## Ordering -- `order_by()`

Sort results with `order_by()`. Use a `-` prefix for descending order
(Django-style shorthand).

In [ ]:
# Ascending by name (alphabetical)
by_name = await User.objects().order_by("name").exec()
print("By name (asc):")
for u in by_name:
    print(f"  {u.name}")

# Descending by age (oldest first)
by_age_desc = await User.objects().order_by("-age").exec()
print("\nBy age (desc):")
for u in by_age_desc:
    print(f"  {u.name}: {u.age}")

## Pagination -- `limit()` and `offset()`

For large result sets (or building paginated APIs), use `limit()` and
`offset()`. They map directly to SurrealQL's `LIMIT` and `START` clauses.

In [ ]:
page_size = 2

# Page 1
page1 = await User.objects().order_by("name").limit(page_size).offset(0).exec()
print(f"Page 1: {[u.name for u in page1]}")

# Page 2
page2 = await User.objects().order_by("name").limit(page_size).offset(page_size).exec()
print(f"Page 2: {[u.name for u in page2]}")

# Page 3
page3 = await User.objects().order_by("name").limit(page_size).offset(page_size * 2).exec()
print(f"Page 3: {[u.name for u in page3]}")

## Selecting Specific Fields -- `select()`

When you only need a subset of fields, `select()` lets you fetch just those
columns. This reduces data transfer and can speed up large queries.

Note: `select()` returns dictionaries, not model instances.

In [ ]:
# Select only name and email (returns dicts)
names_emails = await User.objects().select("name", "email").exec()
print("Names and emails:")
for row in names_emails:
    print(f"  {row}")

## Cleanup

Remove the users table so the database is clean for other notebooks.

In [ ]:
client = await SurrealDBConnectionManager.get_client()
await client.query("REMOVE TABLE users;")
print("Cleanup complete -- users table removed.")